# Preparation

In [4]:
import math
import os
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
sys.path.insert(0,'../../../../') # The simulator package and package needed for analysis 
                                  # (which are imported below) are in a parent folder

# Modified package from https://github.com/BilHim/trafficSimulator
from trafficSimulator import *
# Functions needed for simulation transition calculation
from SimulationAnalysis import utility_summary, compute_utility, update_utility_df

In [ ]:
vehicle_rate = 22

## Simulation setup

### Simulation params setup

In [5]:
# Specify the labels of the road segments in each route
all_routes = [[0, 3], [0, 1], [2, 3], [2, 1]]

v_max = 9
s0 = 2
T = 1
b_max = 2
a_max = 1

# Levels of speed limit for fast and slow roads
# expressed as a proportion of the v_max set above
fast_track_factor, slow_track_factor = 1, 0.33 

# Length of the stop zone
stop_distance = 25 

# Waiting time for fast and slow roads (here set as the same)
fast_wait_time = 4
slow_wait_time = 4

# Total number of vehicles
vehicle_limit = 80

# Store vehicle specs to be input in the simulation package
vehicle_specs = {'v_max': v_max, # Desired speed
                 's0': s0, # Safe bumper-to-bumper distance
                 'T': T, # Time gap
                 'b_max': b_max, # Deceleartion
                 'a_max': a_max, # Acceleration
                 'l' : 3
                }

# Empty dataframe to store the simulation data
df = pd.DataFrame(columns=['Vehicle_label', 'Road_order', 
                           'Total_time', 'Leading_vehicles',
                           'Stopped_time', 'Stop_while_front'])

# Initialize the probabilities for choosing each route for the 1st round;
# each route has an equal chance of being selected
vehicle_preferences = dict(zip(range(vehicle_limit), 
                               [[0.25, 0.25, 0.25, 0.25]]*vehicle_limit))

### Topology setup

In [7]:
# Coordinates of the four corners of the traffic network; all four corners
# lie on either the x or the y axis
left, right = -75*math.sqrt(3), 75*math.sqrt(3)
bottom, top = -75, 75

# Building the road lines with corresponding parameters
left_bottom_outbound = ((left+2, 4), (-5, top-2),
                        slow_track_factor, stop_distance, slow_wait_time)
bottom_right_outbound = ((5, top-2), (right-2, 4),
                         fast_track_factor, stop_distance, fast_wait_time)

left_top_outbound = ((left+2, -4), (-5, bottom+2),
                     fast_track_factor, stop_distance, fast_wait_time)
top_right_outbound = ((5, bottom+2), (right-2, -4),
                      slow_track_factor, stop_distance, slow_wait_time)

## For the connections of the topology, we can represent with concrete lines, but
# the vehicles will not actually go through these routes; 
# they are  for demonstration purposes
connection_top_bottom = ((-1.25, bottom+2), (-1.25, top-2))
connection_bottom_top = ((1.25, top-2), (1.25, bottom+2))

### Utility params setup

In [ ]:
# Exploration factor in UCB alogorithm
delta = 0.25

# Penalty factor
alpha = 0

# Utility interpolation factor between money and time
gamma_mean, gamma_var = 7.5, 0.75

# Utility exponent
eta_mean, eta_var = 1, 0.2

## Function needed for running simulation

In [9]:
 def run_simulation(round_number, 
                    all_routes, 
                    vehicle_limit, 
                    df, 
                    vehicle_preferences,
                    vehicle_rate, 
                    vehicle_specs):
    """
    Executes the simulation and returns certain data of the simulation.
    
    Input:
        round_number (integer): the number of the current simulation
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    global left_bottom_outbound
    global bottom_right_outbound
    global left_top_outbound
    global top_right_outbound
    global connection_top_bottom
    global connection_bottom_top
    
    records = df.copy()
    sim = Simulation({
        'round_number': round_number,
        'all_routes': all_routes, # All possible routes
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'records': records, # Table that will capture the needed vehicle-related info
        'vehicle_preferences': vehicle_preferences
        })

    sim.create_roads([
        ## Key routes
        left_bottom_outbound, # Road #0
        bottom_right_outbound, # Road #1

        left_top_outbound, # Road #2
        top_right_outbound, # Road #3

        connection_top_bottom, # Road #4
        connection_bottom_top, # Road #5

        ## Curved corners
        # Note: in the simulation, the vehicles will not actually go pass these
        # routes; they are more for aesthetic purposes
        *curve_road(left_bottom_outbound[1], 
                    (bottom_right_outbound[0][0], bottom_right_outbound[0][1]+0.01), 
                    (0, top), 16), # Outbound bottom corner

        *curve_road(left_top_outbound[1], 
                    (top_right_outbound[0][0], top_right_outbound[0][1]+0.01), 
                    (0, bottom), 16), # Outbound top corner

        *curve_road(left_bottom_outbound[0], 
                    (left_top_outbound[0][0]+0.01, left_top_outbound[0][1]), 
                    (left, 0), 16), # Outbound left corner

        *curve_road(bottom_right_outbound[1], 
                    (top_right_outbound[1][0]+0.01, top_right_outbound[1][1]), 
                    (right, 0), 16), # Outbound right corner
    ])


    sim.create_gen({
        'vehicle_rate': vehicle_rate, # Rate of generating new vehicles
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'vehicles': vehicle_specs
        })

    # Start simulation
    win = Window(sim)
    win.zoom = 4
    new_records = win.run(steps_per_update=5)
    
    return new_records

# Official start: 1st round

In [10]:
%%time
# Run the 1st round
record_df1 = run_simulation(1, all_routes, vehicle_limit, df, vehicle_preferences,
                             vehicle_rate, vehicle_specs)

CPU times: user 2min 15s, sys: 2.93 s, total: 2min 18s
Wall time: 2min 18s


In [11]:
record_df1

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front
0,0,"[0, 1]",67.45,"[-999, -999]","[0, 0]","[0, 0]"
1,1,"[2, 1]",36.45,"[-999, -999]","[0, 0]","[0, 0]"
2,2,"[2, 3]",68.95,"[1, -999]","[0, 0]","[0, 0]"
3,3,"[0, 3]",97.5,"[0, 10]","[0, 0]","[0, 0]"
4,4,"[0, 1]",77.25,"[3, 14]","[2.8999999999999977, 0]","[2.8999999999999977, 0]"
...,...,...,...,...,...,...
75,75,"[0, 3]",145.45,"[74, 74]","[0, 0]","[0, 0]"
76,76,"[0, 1]",118.2,"[75, 69]","[0, 0]","[0, 0]"
77,77,"[0, 3]",150.9,"[76, 75]","[0, 0]","[0, 0]"
78,78,"[2, 1]",73.9,"[73, 73]","[0, 0]","[0, 0]"


## Simulation Analysis of the 1st round

In [12]:
record_df1['Round_number'] = 1

record_df = record_df1.copy()
record_df.head(10)

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front,Round_number
0,0,"[0, 1]",67.45,"[-999, -999]","[0, 0]","[0, 0]",1
1,1,"[2, 1]",36.45,"[-999, -999]","[0, 0]","[0, 0]",1
2,2,"[2, 3]",68.95,"[1, -999]","[0, 0]","[0, 0]",1
3,3,"[0, 3]",97.5,"[0, 10]","[0, 0]","[0, 0]",1
4,4,"[0, 1]",77.25,"[3, 14]","[2.8999999999999977, 0]","[2.8999999999999977, 0]",1
5,5,"[0, 3]",107.1,"[4, 3]","[0, 0]","[0, 0]",1
6,6,"[0, 1]",86.05,"[5, 18]","[2.4499999999999993, 0]","[2.4499999999999993, 0]",1
7,7,"[0, 1]",86.6,"[6, 6]","[0, 0]","[0, 0]",1
8,8,"[0, 1]",88.4,"[7, 7]","[0, 0]","[0, 0]",1
9,9,"[0, 3]",120.45,"[8, 5]","[0, 0]","[0, 0]",1


### Set up dataframe

In [13]:
utility_df = pd.DataFrame(columns=['Vehicle_label', 'Utilities', 'Probabilities'])

utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
utility_df['Routes_taken'] = [[]] * len(utility_df)
utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
utility_df['Probabilities'] = [[0] * len(all_routes)] * len(utility_df)

### Sample values for the utility exponent $\eta$ and the utillity interpolation factor $\gamma$

In [14]:
etas = []
gammas = []

for i in range(len(utility_df)):
    eta = np.random.normal(eta_mean, eta_var, 1)[0]
    while eta < 0:
        eta = np.random.normal(eta_mean, eta_var, 1)[0]
    etas.append(eta)
    
for i in range(len(utility_df)):
    gamma = np.random.normal(gamma_mean, gamma_var, 1)[0]
    while gamma < 0:
        gamma = np.random.normal(gamma_mean, gamma_var, 1)[0]
    gammas.append(gamma)

utility_df['Gamma'] = gammas
utility_df['Eta'] = etas

utility_df = utility_df.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)            
utility_df.head(5)

,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],7.956701,1.292648
1,1,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],6.687905,1.095124
2,2,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],7.110334,0.629371
3,3,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],7.974230,1.187921
4,4,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],7.513168,1.100663


## Summary of the 1st round

In [15]:
utility_tmp = utility_summary(all_routes, record_df)
utility_df = update_utility_df(all_routes,
                               delta, 
                               alpha, 
                               record_df=record_df,
                               df=utility_df, 
                               utility_tmp=utility_tmp)

display(utility_tmp.head(10))
display(utility_df.head(10))

,Vehicle_label,Road_order,Caused_delay,Count,Total_time
0,0,1,0.00,1,67.45
1,1,3,0.00,1,36.45
2,2,2,0.00,1,68.95
3,3,0,2.90,1,97.50
4,4,1,0.00,1,77.25
5,5,0,2.45,1,107.10
6,6,1,0.00,1,86.05
7,7,1,0.00,1,86.60
8,8,1,0.00,1,88.40
9,9,0,0.00,1,120.45


,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[16.651092223153956, -16.420843878086874, 16.6...","[0.33333333333333287, 1.4451715077443322e-15, ...",[1],7.956701,1.292648
1,1,"[16.651092223153956, 16.651092223153956, 16.65...","[0.3333333333081062, 0.3333333333081062, 0.333...",[3],6.687905,1.095124
2,2,"[16.651092223153956, 16.651092223153956, -0.76...","[0.3333333302882386, 0.3333333302882386, 9.135...",[2],7.110334,0.629371
3,3,"[-11.602100147059588, 16.651092223153956, 16.6...","[1.7892589684666768e-13, 0.3333333333332737, 0...",[0],7.974230,1.187921
4,4,"[16.651092223153956, -7.251750699500148, 16.65...","[0.33333333332871073, 1.3867748669319528e-11, ...",[1],7.513168,1.100663
5,5,"[-10.81868537549709, 16.651092223153956, 16.65...","[3.916570199105223e-13, 0.33333333333320275, 0...",[0],6.832674,1.176540
6,6,"[16.651092223153956, -11.444012743665088, 16.6...","[0.3333333333332635, 2.095702780977871e-13, 0....",[1],7.811886,1.194700
7,7,"[16.651092223153956, -1.3831747618551635, 16.6...","[0.3333333316981184, 4.905644879911916e-09, 0....",[1],8.933191,0.724417
8,8,"[16.651092223153956, -15.850656649811365, 16.6...","[0.3333333333333325, 2.5559276579473426e-15, 0...",[1],6.702279,1.276500
9,9,"[-2.2490835225697854, 16.651092223153956, 16.6...","[2.0636531798688428e-09, 0.33333333264544895, ...",[0],7.670082,0.818494


In [16]:
# Export
record_df.to_csv("Saved_data/Simulation_records/Round_1.csv", index=False)
utility_df.to_csv("Saved_data/Utility_data/Round_1.csv", index=False)

# Run the remaining rounds
Utilities, probabilities are updated after each round and fed into the next round

In [17]:
%%time

# Updated probabilities for the 2nd round:
vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))

rounds = 50

for s in range(2, rounds+1):
    record_tmp = run_simulation(s, all_routes, 
                                vehicle_limit, 
                                df, vehicle_preferences,
                                vehicle_rate, vehicle_specs)
    record_tmp['Round_number'] = s
    
    record_df = pd.concat([record_df, record_tmp], ignore_index=True)
    
    utility_tmp = utility_summary(all_routes, record_df)
    utility_df = update_utility_df(all_routes,
                                   delta, 
                                   alpha, 
                                   record_df=record_df,
                                   df=utility_df, 
                                   utility_tmp=utility_tmp,
                                   round_number=s)
    
    vehicle_preferences = dict(zip(range(vehicle_limit), 
                                   utility_df.Probabilities.to_list()))
    
    # Export
    record_df.to_csv(f"Saved_data/Simulation_records/Round_{s}.csv", index=False)
    utility_df.to_csv(f"Saved_data/Utility_data/Round_{s}.csv", index=False)

CPU times: user 1h 44min 5s, sys: 2min 7s, total: 1h 46min 12s
Wall time: 1h 43min 43s
